In [1]:
# import packages

import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('../mock_resq.db')

In [3]:
query_orders = "SELECT * FROM orders"
orders = pd.read_sql_query(query_orders, conn)

query_providers = "SELECT * FROM providers"
providers = pd.read_sql_query(query_providers, conn)

query_users = "SELECT * FROM users"
users = pd.read_sql_query(query_users, conn)

# Exploratory data analysis (EDA)
- In this section, we examine various aspects of the data to identify duplicates, missing values, and other relevant information

In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299971 entries, 0 to 299970
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          299971 non-null  int64 
 1   createdAt   299971 non-null  object
 2   userId      299971 non-null  int64 
 3   quantity    299971 non-null  object
 4   refunded    299971 non-null  object
 5   currency    299971 non-null  object
 6   sales       299971 non-null  object
 7   providerId  299971 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 18.3+ MB


In [13]:
providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4337 entries, 0 to 4336
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4337 non-null   int64 
 1   defaultOfferType  4337 non-null   object
 2   country           4337 non-null   object
 3   registeredDate    4337 non-null   object
dtypes: int64(1), object(3)
memory usage: 135.7+ KB


In [14]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358366 entries, 0 to 358365
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              358366 non-null  int64 
 1   country         358366 non-null  object
 2   registeredDate  358366 non-null  object
dtypes: int64(1), object(2)
memory usage: 8.2+ MB


In [15]:
duplicates = orders[orders.duplicated()]
print("Number of orders duplications:", len(duplicates))

Number of orders duplications: 0


In [16]:
duplicates = users[users.duplicated()]
print("Number of users duplications:", len(duplicates))

Number of users duplications: 0


In [17]:
duplicates = providers[providers.duplicated()]
print("Number of providers duplications:", len(duplicates))

Number of providers duplications: 0


In [18]:
# Check for the missing values
users.isnull().any()

id                False
country           False
registeredDate    False
dtype: bool

In [19]:
# Check for the missing values
orders.isnull().any()

id            False
createdAt     False
userId        False
quantity      False
refunded      False
currency      False
sales         False
providerId    False
dtype: bool

In [20]:
# Check for the missing values
providers.isnull().any()

id                  False
defaultOfferType    False
country             False
registeredDate      False
dtype: bool

We've examined the data in preparation for upcoming analysis, particularly regarding problem two. Fortunately, the data appears to be free of NaNs, missing values, or duplications

In [26]:
orders_regDate = pd.merge(orders, users[['id', 'registeredDate']], left_on='userId', right_on='id')
orders_regDate = orders_regDate.drop('id_y', axis=1)
orders_regDate = orders_regDate.rename(columns={'id_x': 'id'})

In [29]:
orders_regDate["registeredDate"] = pd.to_datetime(orders_regDate["registeredDate"])
orders_regDate["registeredDate"] = orders_regDate.registeredDate.dt.year

In [37]:
orders_regDate_offer = pd.merge(orders_regDate, providers[['id', 'defaultOfferType']], left_on='providerId', right_on='id')
orders_regDate_offer = orders_regDate_offer.drop('id_y', axis=1)
orders_regDate_offer = orders_regDate_offer.rename(columns={'id_x': 'id'})

In [39]:
orders_regDate_offer['createdAt'] = pd.to_datetime(orders_regDate_offer['createdAt'])
orders_regDate_offer['year_month'] = orders_regDate_offer['createdAt'].dt.to_period('M')

In [42]:
orders_regDate_offer['year_month'] = orders_regDate_offer['year_month'].astype(str)

In [43]:
conn = sqlite3.connect('orders_regDate_offer.db')

# Write the DataFrame to the database
orders_regDate_offer.to_sql('orders_regDate_offer', conn, if_exists='replace')

# Close the connection
conn.close()